# Cali house prices prediction
---
**Name**: Francisco Aarón Ortega Anguiano

## Dataset columns:
- **longitude**: A measure of how far west a house is; a higher value is farther west.
- **latitude**: A measure of how far north a house is; a higher value is farther north.  
- **housingMedianAge**: Median age of a house within a block; a lower number is a newer building.
- **totalRooms**: Total number of rooms within a block.
- **totalBedrooms**: Total number of bedrooms within a block.
- **population**: Total number of people residing within a block.
- **households**: Total number of households (groups of people residing together) per block.
- **medianIncome**: Median income for households (in tens of thousands of US Dollars).
- **medianHouseValue**: Median house value for households (in US Dollars).
- **oceanProximity**: Location of the house relative to ocean/sea.

### Import important libraries and populate the SQL housing_raw table.

In [2]:
import pandas as pd
from lib.db_engine import DatabaseEngine

required_vars = [
    "DB_USER",
    "DB_PASSWORD",
    "DB_NAME"
]

db_engine = DatabaseEngine(required_vars)

engine = db_engine.generate_engine()

# connect to the database and printing connection successful if connected
with engine.connect() as conn:
    print("Connection successful")

# read the csv file with the dataset
df = pd.read_csv("./db/housing.csv")

# populate the housing_raw table using or engine as connection
df.to_sql("housing_raw", engine, if_exists="replace", index=False)

Connection successful


640

In [ ]:
# check that it was correctly populated
pd.read_sql("SELECT * FROM housing_raw LIMIT 5", engine)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
print(f"data shape: {df.shape} \n")
df.info()

data shape: (20640, 10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [ ]:
# check for null values in the dataframe
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [ ]:
# check if it matches in our database
pd.read_sql("SELECT COUNT(*) FROM housing_raw WHERE total_bedrooms IS NULL", engine)

,count
0,207


Checking the shape of the df and the info we can see that the `total_bedrooms` column has 207 missing values, we are going to impute this with the `AVG` of that same colum with a simple SQL query in the following [file](./3_ImputeAvgTotalBedrooms.sql).

In [ ]:
# check if the imputation was done correctly
pd.read_sql("SELECT COUNT(*) FROM housing_raw WHERE total_bedrooms IS NULL", engine)

,count
0,0


In [ ]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000
